# Data labeling example
This notebook shows how to use EDSL to automate data labeling and content analysis. Using a dataset of mock customer service tickets as an example, we design a data labeling task as a survey of questions about the tickets that we administer to a language model, generating a summary dataset.

The steps are:
1. Technical setup<br>
2. Identifying data for review<br>
3. Constructing questions about the data<br>
4. Delivering the questions to language models<br>
5. Analyzing the responses<br>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/data_labeling_example.ipynb)

### Technical setup
Before running the code below please see instructions on:

* [Installing the EDSL package](https://docs.expectedparrot.com/en/latest/installation.html) and
* [Storing API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use

We also have a [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) about EDSL basics.

In [1]:
# ! pip install edsl

## Selecting data for review
First we identify some data for review. Data can be created using the EDSL tools or imported from other sources. For purposes of this demo we import a set of hypothetical customer tickets for a transportation app:

In [2]:
tickets = [
    "I just realized I left my phone in the car on my last ride. Can you help me get it back?",
    "I'm unhappy with my recent experience. The driver was very rude and unprofessional.",
    "I was charged more than the estimated fare for my trip yesterday. Can you explain why?",
    "The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.",
    "My driver took a longer route than necessary, resulting in a higher fare. I request a fare adjustment.",
    "I had a great experience with my driver today! Very friendly and efficient service.",
    "I'm concerned about the vehicle's cleanliness. It was not up to the standard I expect.",
    "The app keeps crashing every time I try to book a ride. Please fix this issue.",
    "My driver was exceptional - safe driving, polite, and the car was spotless. Kudos!",
    "I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.",
    "The driver refused to follow my preferred route, which is shorter. I'm not satisfied with the service.",
    "Impressed with the quick response to my ride request and the driver's professionalism.",
    "I was charged for a ride I never took. Please refund me as soon as possible.",
    "The promo code I tried to use didn't work. Can you assist with this?",
    "There was a suspicious smell in the car, and I'm worried about hygiene standards.",
    "My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",
    "The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.",
    "I want to compliment my driver's excellent navigation and time management during rush hour.",
    "The vehicle didn't match the description in the app. It was confusing and concerning.",
    "I faced an issue with payment processing after my last ride. Can you look into this?",
]

## Constructing questions about the data
Next we create some questions about the data. EDSL provides classes for a variety of common question types, including multiple choice, free text, checkbox, linear scale and many others. We import the question type classes that we want to use and then construct questions in the relevant templates. We use a <b><i>{{ placeholder }}</b></i> in each question text in order to parameterize the questions with the individual ticket contents. For more details about constructing questions, please see the <a href="https://docs.expectedparrot.com/en/latest/questions.html">Questions</a> section of the docs.

In [3]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionList, QuestionYesNo, QuestionLinearScale

In [4]:
question_issues = QuestionCheckBox(
    question_name = "issues",
    question_text = "Check all of the issues mentioned in this ticket: {{ ticket }}",
    question_options = [
        "safety", 
        "cleanliness", 
        "driver performance", 
        "GPS/route", 
        "lost item", 
        "other"]
)

In [5]:
question_primary_issue = QuestionFreeText(
    question_name = "primary_issue",
    question_text = "What is the primary issue in this ticket? Ticket: {{ ticket }}"
)

In [6]:
question_accident = QuestionMultipleChoice(
    question_name = "accident",
    question_text = "If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
    question_options = ["Yes", "No", "Not applicable"]
)

In [7]:
question_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the sentiment of this ticket? Ticket: {{ ticket }}",
    question_options = [
        "Very positive",
        "Somewhat positive",
        "Neutral",
        "Somewhat negative",
        "Very negative"
    ]
)

In [8]:
question_refund = QuestionYesNo(
    question_name = "refund",
    question_text = "Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}"
)

In [9]:
question_priority = QuestionLinearScale(
    question_name = "priority",
    question_text = "On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}",
    question_options = [0,1,2,3,4,5],
    option_labels = {0: "Lowest", 5: "Highest"}
)

## Building a survey
We combine the questions into a survey in order to administer them together:

In [10]:
from edsl import Survey

survey = Survey(
    questions = [
        question_issues,
        question_primary_issue,
        question_accident,
        question_sentiment,
        question_refund,
        question_priority
    ]
)

(Survey questions are administered asynchronously by default. To learn about adding conditional logic such as skip/stop rules and targeted memory, please see the <a href="https://docs.expectedparrot.com/en/latest/surveys.html">Surveys</a> section of the docs.)

We can review our questions in a readable format, or export them as a survey to use with human respondents or at other survey platforms:

In [11]:
survey.print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Questions                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                        ┃ Options                              ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ issues        │ checkbox      │ Check all of the issues mentioned in │ safety, cleanliness, driver          │ │
│ │               │               │ this ticket: {{ ticket }}            │ performance, GPS/route, lost item,   │ │
│ │               │               │                                      │ other                                │ │
│ └───────────────┴───────────────┴──────────────────────────────────────┴──────────────────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓    │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                  ┃ Options ┃    │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩    │
│ │ primary_issue │ free_text     │ What is the primary issue in this ticket? Ticket: {{ ticket }} │ None    │    │
│ └───────────────┴───────────────┴────────────────────────────────────────────────────────────────┴─────────┘    │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type   ┃ Question Text                                   ┃ Options                 ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ accident      │ multiple_choice │ If the primary issue in this ticket is safety,  │ Yes, No, Not applicable │ │
│ │               │                 │ was there an accident where someone was hurt?   │                         │ │
│ │               │                 │ Ticket: {{ ticket }}                            │                         │ │
│ └───────────────┴─────────────────┴─────────────────────────────────────────────────┴─────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type   ┃ Question Text                       ┃ Options                             ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ sentiment     │ multiple_choice │ What is the sentiment of this       │ Very positive, Somewhat positive,   │ │
│ │               │                 │ ticket? Ticket: {{ ticket }}        │ Neutral, Somewhat negative, Very    │ │
│ │               │                 │                                     │ negative                            │ │
│ └───────────────┴─────────────────┴─────────────────────────────────────┴─────────────────────────────────────┘ │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ │
│ ┃ Question Name ┃ Question Type ┃ Question Text                                                     ┃ Options ┃ │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ │
│ │ refund        │ yes_no        │ Does the customer ask for a refund in this ticket? Ticket: {{     │ Yes, No │ │
│ │               │               │ ticket }}                                                         │         │ │
│ └───────────────┴───────────────┴─────────────────────

In [12]:
survey

{
  "questions": [
    {
      "question_name": "issues",
      "question_text": "Check all of the issues mentioned in this ticket: {{ ticket }}",
      "min_selections": null,
      "max_selections": null,
      "question_options": [
        "safety",
        "cleanliness",
        "driver performance",
        "GPS/route",
        "lost item",
        "other"
      ],
      "question_type": "checkbox"
    },
    {
      "question_name": "primary_issue",
      "question_text": "What is the primary issue in this ticket? Ticket: {{ ticket }}",
      "question_type": "free_text"
    },
    {
      "question_name": "accident",
      "question_text": "If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
      "question_options": [
        "Yes",
        "No",
        "Not applicable"
      ],
      "question_type": "multiple_choice"
    },
    {
      "question_name": "sentiment",
      "question_text": "What is the sentiment of this ticket? Ticket: {{ ticket }}",
      "question_options": [
        "Very positive",
        "Somewhat positive",
        "Neutral",
        "Somewhat negative",
        "Very negative"
      ],
      "question_type": "multiple_choice"
    },
    {
      "question_name": "refund",
      "question_text": "Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}",
      "question_options": [
        "Yes",
        "No"
      ],
      "question_type": "yes_no"
    },
    {
      "question_name": "priority",
      "question_text": "On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}",
      "question_options": [
        0,
        1,
        2,
        3,
        4,
        5
      ],
      "option_labels": {
        "0": "Lowest",
        "5": "Highest"
      },
      "question_type": "linear_scale"
    }
  ],
  "name": null,
  "memory_plan": {
    "survey_question_names": [
      "issues",
      "primary_issue",
      "accident",
      "sentiment",
      "refund",
      "priority"
    ],
    "survey_question_texts": [
      "Check all of the issues mentioned in this ticket: {{ ticket }}",
      "What is the primary issue in this ticket? Ticket: {{ ticket }}",
      "If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
      "What is the sentiment of this ticket? Ticket: {{ ticket }}",
      "Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}",
      "On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}"
    ],
    "data": {}
  },
  "rule_collection": {
    "rules": [
      {
        "current_q": 0,
        "expression": "True",
        "next_q": 1,
        "priority": -1,
        "question_name_to_index": {
          "issues": 0
        }
      },
      {
        "current_q": 1,
        "expression": "True",
        "next_q": 2,
        "priority": -1,
        "question_name_to_index": {
          "issues": 0,
          "primary_issue": 1
        }
      },
      {
        "current_q": 2,
        "expression": "True",
        "next_q": 3,
        "priority": -1,
        "question_name_to_index": {
          "issues": 0,
          "primary_issue": 1,
          "accident": 2
        }
      },
      {
        "current_q": 3,
        "expression": "True",
        "next_q": 4,
        "priority": -1,
        "question_name_to_index": {
          "issues": 0,
          "primary_issue": 1,
          "accident": 2,
          "sentiment": 3
        }
      },
      {
        "current_q": 4,
        "expression": "True",
        "next_q": 5,
        "priority": -1,
        "question_name_to_index": {
          "issues": 0,
          "primary_issue": 1,
          "accident": 2,
          "sentiment": 3,
          "refund": 4
        }
      },
      {
        "current_q": 5,
        "expression": "True",
        "next_q": 6,
        "priority": -1,
        "question_name_to_index": {
 

{
  "question_name": "issues",
  "question_text": "Check all of the issues mentioned in this ticket: {{ ticket }}",
  "min_selections": null,
  "max_selections": null,
  "question_options": [
    "safety",
    "cleanliness",
    "driver performance",
    "GPS/route",
    "lost item",
    "other"
  ],
  "question_type": "checkbox"
}

{
  "question_name": "primary_issue",
  "question_text": "What is the primary issue in this ticket? Ticket: {{ ticket }}",
  "question_type": "free_text"
}

{
  "question_name": "accident",
  "question_text": "If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}",
  "question_options": [
    "Yes",
    "No",
    "Not applicable"
  ],
  "question_type": "multiple_choice"
}

{
  "question_name": "sentiment",
  "question_text": "What is the sentiment of this ticket? Ticket: {{ ticket }}",
  "question_options": [
    "Very positive",
    "Somewhat positive",
    "Neutral",
    "Somewhat negative",
    "Very negative"
  ],
  "question_type": "multiple_choice"
}

{
  "question_name": "refund",
  "question_text": "Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}",
  "question_options": [
    "Yes",
    "No"
  ],
  "question_type": "yes_no"
}

{
  "question_name": "priority",
  "question_text": "On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}",
  "question_options": [
    0,
    1,
    2,
    3,
    4,
    5
  ],
  "option_labels": {
    "0": "Lowest",
    "5": "Highest"
  },
  "question_type": "linear_scale"
}

Survey(questions=[QuestionCheckBox(question_name = 'issues', question_text = 'Check all of the issues mentioned in this ticket: {{ ticket }}', min_selections = None, max_selections = None, question_options = ['safety', 'cleanliness', 'driver performance', 'GPS/route', 'lost item', 'other']), QuestionFreeText(question_name = 'primary_issue', question_text = 'What is the primary issue in this ticket? Ticket: {{ ticket }}'), QuestionMultipleChoice(question_name = 'accident', question_text = 'If the primary issue in this ticket is safety, was there an accident where someone was hurt? Ticket: {{ ticket }}', question_options = ['Yes', 'No', 'Not applicable']), QuestionMultipleChoice(question_name = 'sentiment', question_text = 'What is the sentiment of this ticket? Ticket: {{ ticket }}', question_options = ['Very positive', 'Somewhat positive', 'Neutral', 'Somewhat negative', 'Very negative']), QuestionYesNo(question_name = 'refund', question_text = 'Does the customer ask for a refund in this ticket? Ticket: {{ ticket }}', question_options = ['Yes', 'No']), QuestionLinearScale(question_name = 'priority', question_text = 'On a scale from 0 to 5, what is the priority level of this ticket? Ticket: {{ ticket }}', question_options = [0, 1, 2, 3, 4, 5], option_labels = {0: 'Lowest', 5: 'Highest'})], name=None)

## Designing AI agents
A key feature of EDSL is the ability to create personas for AI agents that the language models are prompted to use in generating responses to the questions. This is done by passing a dictionary of traits to Agent objects:

In [13]:
from edsl import Agent

agent = Agent(traits = {
    "persona": "You are an expert customer service agent.",
    "years_experience": 15
})

## Selecting language models
EDSL allows us to select the language models to use in generating results. To see all available models:

In [14]:
from edsl import Model

Model.available()

[['01-ai/Yi-34B-Chat', 'deep_infra', 0],
 ['Austism/chronos-hermes-13b-v2', 'deep_infra', 1],
 ['Gryphe/MythoMax-L2-13b', 'deep_infra', 2],
 ['HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1', 'deep_infra', 3],
 ['Phind/Phind-CodeLlama-34B-v2', 'deep_infra', 4],
 ['bigcode/starcoder2-15b', 'deep_infra', 5],
 ['claude-3-haiku-20240307', 'anthropic', 6],
 ['claude-3-opus-20240229', 'anthropic', 7],
 ['claude-3-sonnet-20240229', 'anthropic', 8],
 ['codellama/CodeLlama-34b-Instruct-hf', 'deep_infra', 9],
 ['codellama/CodeLlama-70b-Instruct-hf', 'deep_infra', 10],
 ['cognitivecomputations/dolphin-2.6-mixtral-8x7b', 'deep_infra', 11],
 ['databricks/dbrx-instruct', 'deep_infra', 12],
 ['deepinfra/airoboros-70b', 'deep_infra', 13],
 ['gemini-pro', 'google', 14],
 ['google/gemma-1.1-7b-it', 'deep_infra', 15],
 ['gpt-3.5-turbo', 'openai', 16],
 ['gpt-3.5-turbo-0125', 'openai', 17],
 ['gpt-3.5-turbo-0301', 'openai', 18],
 ['gpt-3.5-turbo-0613', 'openai', 19],
 ['gpt-3.5-turbo-1106', 'openai', 20],
 ['gp

Here we select GPT 4 (which is also used by default if no model is specified for a survey):

In [15]:
model = Model("gpt-4-1106-preview")

## Conducting the analysis
With our data and questions we're now ready to package our survey and deliver it to an AI. We do this by inserting the contents of each ticket into each question as an independent "scenario" for review, and then running the survey.

In [16]:
from edsl import Scenario

scenarios = [Scenario({"ticket":t}) for t in tickets] 

We run the survey by chaining the components with the <i>by</i> method and then calling the <i>run</i> method:

In [17]:
results = survey.by(scenarios).by(agent).by(model).run()

## Inspecting the results
EDSL comes with built-in methods for analyzing results. For more details on working with results, please see the <a href="https://docs.expectedparrot.com/en/latest/results.html">Results</a> section of the docs.

To inspect the components of results:

In [18]:
results.columns

['agent.agent_name',
 'agent.persona',
 'agent.years_experience',
 'answer.accident',
 'answer.accident_comment',
 'answer.issues',
 'answer.issues_comment',
 'answer.primary_issue',
 'answer.priority',
 'answer.priority_comment',
 'answer.refund',
 'answer.refund_comment',
 'answer.sentiment',
 'answer.sentiment_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.accident_system_prompt',
 'prompt.accident_user_prompt',
 'prompt.issues_system_prompt',
 'prompt.issues_user_prompt',
 'prompt.primary_issue_system_prompt',
 'prompt.primary_issue_user_prompt',
 'prompt.priority_system_prompt',
 'prompt.priority_user_prompt',
 'prompt.refund_system_prompt',
 'prompt.refund_user_prompt',
 'prompt.sentiment_system_prompt',
 'prompt.sentiment_user_prompt',
 'raw_model_response.accident_raw_model_response',
 'raw_model_response.issues_

We can select just the responses to the questions to display:

In [19]:
results.select('ticket', 'issues', 'primary_issue', 'accident', 'sentiment', 'refund', 'priority').print()

scenario.ticket,answer.issues,answer.primary_issue,answer.accident,answer.sentiment,answer.refund,answer.priority
I faced an issue with payment processing after my last ride. Can you look into this?,['other'],The primary issue in the ticket is a problem with payment processing after the customer completed their last ride.,Not applicable,Somewhat negative,No,4
I just realized I left my phone in the car on my last ride. Can you help me get it back?,['lost item'],The primary issue in this ticket is that the customer has left their phone in the car during their last ride and is seeking assistance to retrieve it.,No,Somewhat negative,No,4
Impressed with the quick response to my ride request and the driver's professionalism.,['driver performance'],The primary issue in the ticket is not an issue but positive feedback regarding the quick response to the customer's ride request and the driver's professionalism.,Not applicable,Very positive,No,0
I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.,"['safety', 'driver performance']",The primary issue in the ticket is that the customer felt unsafe due to the driver's erratic behavior during the ride.,No,Very negative,No,5
The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.,['GPS/route'],"The primary issue in this ticket is that the app's GPS is providing inaccurate directions, which led to the driver being directed to an incorrect pick-up location.",Not applicable,Somewhat negative,No,4
"The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.","['safety', 'driver performance']","The primary issue in the ticket is that the car seat provided was not properly installed, which led to concerns about the child's safety. The customer is requesting that drivers receive proper training in car seat installation.",No,Somewhat negative,No,5
The promo code I tried to use didn't work. Can you assist with this?,['other'],"The primary issue is that the customer's promo code is not working, and they are requesting assistance to resolve this problem.",No,Somewhat negative,No,2
I was charged more than the estimated fare for my trip yesterday. Can you explain why?,['other'],The primary issue in the ticket is that the customer was charged an amount exceeding the estimated fare for their trip and is seeking an explanation for the discrepancy.,Not applicable,Somewhat negative,No,3
"There was a suspicious smell in the car, and I'm worried about hygiene standards.","['safety', 'cleanliness']","The primary issue in the ticket is a concern regarding a suspicious smell in the car, which has raised worries about the vehicle's hygiene standards.",Not applicable,Somewhat negative,No,4
"My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",['driver performance'],There is no issue reported in this ticket. The customer is expressing appreciation for the considerate service provided by the driver.,No,Very positive,No,0


We can apply some lables to our table:

In [20]:
(results
 .select('ticket', 'issues', 'primary_issue', 'accident', 'sentiment', 'refund', 'priority')
 .print(pretty_labels={
     'scenario.ticket':'Ticket',
     'answer.issues':'Issues',
     'answer.primary_issue':'Primary issue',
     'answer.accident':'Accident',
     'answer.sentiment':'Sentiment',
     'answer.refund':'Refund request',
     'answer.priority':'Priority'})
)

Ticket,Issues,Primary issue,Accident,Sentiment,Refund request,Priority
I faced an issue with payment processing after my last ride. Can you look into this?,['other'],The primary issue in the ticket is a problem with payment processing after the customer completed their last ride.,Not applicable,Somewhat negative,No,4
I just realized I left my phone in the car on my last ride. Can you help me get it back?,['lost item'],The primary issue in this ticket is that the customer has left their phone in the car during their last ride and is seeking assistance to retrieve it.,No,Somewhat negative,No,4
Impressed with the quick response to my ride request and the driver's professionalism.,['driver performance'],The primary issue in the ticket is not an issue but positive feedback regarding the quick response to the customer's ride request and the driver's professionalism.,Not applicable,Very positive,No,0
I felt unsafe during my ride due to the driver's erratic behavior. This needs to be addressed immediately.,"['safety', 'driver performance']",The primary issue in the ticket is that the customer felt unsafe due to the driver's erratic behavior during the ride.,No,Very negative,No,5
The app's GPS seems inaccurate. It directed the driver to the wrong pick-up location.,['GPS/route'],"The primary issue in this ticket is that the app's GPS is providing inaccurate directions, which led to the driver being directed to an incorrect pick-up location.",Not applicable,Somewhat negative,No,4
"The car seat provided was not properly installed, and I felt my child was at risk. Please ensure driver training.","['safety', 'driver performance']","The primary issue in the ticket is that the car seat provided was not properly installed, which led to concerns about the child's safety. The customer is requesting that drivers receive proper training in car seat installation.",No,Somewhat negative,No,5
The promo code I tried to use didn't work. Can you assist with this?,['other'],"The primary issue is that the customer's promo code is not working, and they are requesting assistance to resolve this problem.",No,Somewhat negative,No,2
I was charged more than the estimated fare for my trip yesterday. Can you explain why?,['other'],The primary issue in the ticket is that the customer was charged an amount exceeding the estimated fare for their trip and is seeking an explanation for the discrepancy.,Not applicable,Somewhat negative,No,3
"There was a suspicious smell in the car, and I'm worried about hygiene standards.","['safety', 'cleanliness']","The primary issue in the ticket is a concern regarding a suspicious smell in the car, which has raised worries about the vehicle's hygiene standards.",Not applicable,Somewhat negative,No,4
"My driver was very considerate, especially helping me with my luggage. Appreciate the great service!",['driver performance'],There is no issue reported in this ticket. The customer is expressing appreciation for the considerate service provided by the driver.,No,Very positive,No,0


EDSL also comes with methods for accessing results as a dataframe or SQL table:

In [21]:
df = results.to_pandas()
df

,agent.agent_name,agent.persona,agent.years_experience,answer.accident,answer.accident_comment,answer.issues,answer.issues_comment,answer.primary_issue,answer.priority,answer.priority_comment,...,prompt.refund_user_prompt,prompt.sentiment_system_prompt,prompt.sentiment_user_prompt,raw_model_response.accident_raw_model_response,raw_model_response.issues_raw_model_response,raw_model_response.primary_issue_raw_model_response,raw_model_response.priority_raw_model_response,raw_model_response.refund_raw_model_response,raw_model_response.sentiment_raw_model_response,scenario.ticket
0,Agent_0,You are an expert customer service agent.,15,Not applicable,The issue described in the ticket pertains to ...,['other'],The issue mentioned relates to payment process...,The primary issue in the ticket is a problem w...,4,Payment processing issues are high priority as...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCo4GC6mqjOczf2BYwCMVrywlWq...,{'id': 'chatcmpl-9JqCowmmXTBypWExu1MNde4sW6Dz0...,{'id': 'chatcmpl-9JqCodMvs5dlo9LrC6iCeHKu4SAtJ...,{'id': 'chatcmpl-9JqCoVxBHTV0RRLTP0Ahgyh7Cwu6S...,{'id': 'chatcmpl-9JqCojMJXAzXAVj2EtjmQzs4viDVs...,{'id': 'chatcmpl-9JqCoxT4YzzxHKPKi6v9551e0fp7P...,I faced an issue with payment processing after...
1,Agent_0,You are an expert customer service agent.,15,No,The primary issue is about a lost phone in the...,['lost item'],The issue mentioned in the ticket pertains to ...,The primary issue in this ticket is that the c...,4,Losing a phone is a high-priority issue due to...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCouMzSLXnBGLkMsw2gnJBywVRq...,{'id': 'chatcmpl-9JqCopROBXJGS9Rlt4YXRM3StgcQK...,{'id': 'chatcmpl-9JqCoMcfIG3nU3w95eWFNASLUmGrg...,{'id': 'chatcmpl-9JqCobUROGwdZAKbKDEEU4GzbQZbm...,{'id': 'chatcmpl-9JqCoEf7yWD3s1pTTx7yMBQob1gRC...,{'id': 'chatcmpl-9JqCoWmkl56RAyMsAbv3Mbs8SLFCB...,I just realized I left my phone in the car on ...
2,Agent_0,You are an expert customer service agent.,15,Not applicable,The ticket does not mention any accident or sa...,['driver performance'],The ticket mentions a positive remark regardin...,The primary issue in the ticket is not an issu...,0,This ticket is a positive feedback and does no...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCp2YYwRiiRdpH74iWIT2nHKhnt...,{'id': 'chatcmpl-9JqCohhr26yla77WpHyGPFU86trfb...,{'id': 'chatcmpl-9JqCokcVVD7a1tqd7lA8n8fp5y2Xf...,{'id': 'chatcmpl-9JqCoCbxIFZBu1KPy1YMwZxcAcH7w...,{'id': 'chatcmpl-9JqCofpMga3uTq73LpCtNYFZlxrea...,{'id': 'chatcmpl-9JqCo7FpwD2OlPhge7Mlvkf1HP5sa...,Impressed with the quick response to my ride r...
3,Agent_0,You are an expert customer service agent.,15,No,The ticket mentions a feeling of unsafety due ...,"['safety', 'driver performance']",The ticket mentions feeling unsafe due to the ...,The primary issue in the ticket is that the cu...,5,Safety concerns are of the utmost priority and...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCo7l4rgO1id057yUqpBz6I8vYe...,{'id': 'chatcmpl-9JqCoFojGyWBxfBgj8LwanpDfxYYz...,{'id': 'chatcmpl-9JqCospwZu1ctUh3RWMee3s7dMsKJ...,{'id': 'chatcmpl-9JqCoxAZ6LAsb6iABByd2WV4pbpBE...,{'id': 'chatcmpl-9JqCotqmwevjUBX4a0s2rViyJsw4G...,{'id': 'chatcmpl-9JqCoUhCJ5B1BvIwof30WS3gudqI6...,I felt unsafe during my ride due to the driver...
4,Agent_0,You are an expert customer service agent.,15,Not applicable,The ticket does not mention any accident or in...,['GPS/route'],The issue reported is related to the GPS or ro...,The primary issue in this ticket is that the a...,4,GPS inac

We can also access results as a SQL table:

In [22]:
results.sql("select * from self", shape="wide")

,agent.agent_name,agent.persona,agent.years_experience,answer.accident,answer.accident_comment,answer.issues,answer.issues_comment,answer.primary_issue,answer.priority,answer.priority_comment,...,prompt.refund_user_prompt,prompt.sentiment_system_prompt,prompt.sentiment_user_prompt,raw_model_response.accident_raw_model_response,raw_model_response.issues_raw_model_response,raw_model_response.primary_issue_raw_model_response,raw_model_response.priority_raw_model_response,raw_model_response.refund_raw_model_response,raw_model_response.sentiment_raw_model_response,scenario.ticket
0,Agent_0,You are an expert customer service agent.,15,Not applicable,The issue described in the ticket pertains to ...,['other'],The issue mentioned relates to payment process...,The primary issue in the ticket is a problem w...,4,Payment processing issues are high priority as...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCo4GC6mqjOczf2BYwCMVrywlWq...,{'id': 'chatcmpl-9JqCowmmXTBypWExu1MNde4sW6Dz0...,{'id': 'chatcmpl-9JqCodMvs5dlo9LrC6iCeHKu4SAtJ...,{'id': 'chatcmpl-9JqCoVxBHTV0RRLTP0Ahgyh7Cwu6S...,{'id': 'chatcmpl-9JqCojMJXAzXAVj2EtjmQzs4viDVs...,{'id': 'chatcmpl-9JqCoxT4YzzxHKPKi6v9551e0fp7P...,I faced an issue with payment processing after...
1,Agent_0,You are an expert customer service agent.,15,No,The primary issue is about a lost phone in the...,['lost item'],The issue mentioned in the ticket pertains to ...,The primary issue in this ticket is that the c...,4,Losing a phone is a high-priority issue due to...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCouMzSLXnBGLkMsw2gnJBywVRq...,{'id': 'chatcmpl-9JqCopROBXJGS9Rlt4YXRM3StgcQK...,{'id': 'chatcmpl-9JqCoMcfIG3nU3w95eWFNASLUmGrg...,{'id': 'chatcmpl-9JqCobUROGwdZAKbKDEEU4GzbQZbm...,{'id': 'chatcmpl-9JqCoEf7yWD3s1pTTx7yMBQob1gRC...,{'id': 'chatcmpl-9JqCoWmkl56RAyMsAbv3Mbs8SLFCB...,I just realized I left my phone in the car on ...
2,Agent_0,You are an expert customer service agent.,15,Not applicable,The ticket does not mention any accident or sa...,['driver performance'],The ticket mentions a positive remark regardin...,The primary issue in the ticket is not an issu...,0,This ticket is a positive feedback and does no...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCp2YYwRiiRdpH74iWIT2nHKhnt...,{'id': 'chatcmpl-9JqCohhr26yla77WpHyGPFU86trfb...,{'id': 'chatcmpl-9JqCokcVVD7a1tqd7lA8n8fp5y2Xf...,{'id': 'chatcmpl-9JqCoCbxIFZBu1KPy1YMwZxcAcH7w...,{'id': 'chatcmpl-9JqCofpMga3uTq73LpCtNYFZlxrea...,{'id': 'chatcmpl-9JqCo7FpwD2OlPhge7Mlvkf1HP5sa...,Impressed with the quick response to my ride r...
3,Agent_0,You are an expert customer service agent.,15,No,The ticket mentions a feeling of unsafety due ...,"['safety', 'driver performance']",The ticket mentions feeling unsafe due to the ...,The primary issue in the ticket is that the cu...,5,Safety concerns are of the utmost priority and...,...,{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9JqCo7l4rgO1id057yUqpBz6I8vYe...,{'id': 'chatcmpl-9JqCoFojGyWBxfBgj8LwanpDfxYYz...,{'id': 'chatcmpl-9JqCospwZu1ctUh3RWMee3s7dMsKJ...,{'id': 'chatcmpl-9JqCoxAZ6LAsb6iABByd2WV4pbpBE...,{'id': 'chatcmpl-9JqCotqmwevjUBX4a0s2rViyJsw4G...,{'id': 'chatcmpl-9JqCoUhCJ5B1BvIwof30WS3gudqI6...,I felt unsafe during my ride due to the driver...
4,Agent_0,You are an expert customer service agent.,15,Not applicable,The ticket does not mention any accident or in...,['GPS/route'],The issue reported is related to the GPS or ro...,The primary issue in this ticket is that the a...,4,GPS inac

To export results to a CSV file:

In [23]:
results.to_csv("data_labeling_example.csv")